## 1. Get dataset (Business & data understanding)

>가장 먼저, pdf에서 제공받은 홈페이지에서 dataset을 가져와 읽어온다.

나는 포르투갈에서 진행한 은행 캠페인의 고객 dataset을 받아왔다.  
Classfication에 적합한 data이며, 해당 dataset에서 각 열의 의미는 아래와 같다.  

- age : 고객의 나이
- job : 고객의 직업
- marital : 기혼 여부
- education : 교육 여부 (unknown, primary, secondary, teritary)
- default : 카드 요금 체납 여부
- balance : 연 평균 잔고 (유로 단위)
- housing : 주택 대출 여부
- loan : 개인 대출 여부
- contact : 연락 수신 유형 (unknown, telephone, cellular)
- day : 해당 달의 마지막 연락(contact) 일자
- month : 해당 연도의 마지막 연락(contact) 달
- duration : 마지막 contact 기간 (초 단위)
- campaign : 해당 campaign이 진행되는 동안, 고객에게 간 연락(contact)의 횟수
- pdays : 이전 campaign의 연락을 받은 뒤로 며칠이 지났는지?
  - -1은, 이전에 연락이 간 적이 없음을 의미한다.
- previous : 해당 campaign이 진행되기 전 고객에게 간 연락(contact)의 횟수
- poutcome : 이전 campaign에 대한 결과 (unknwon, other, failure, success)
- y : 위의 입력에 대한 결과 값이다.
  - 위의 입력에 대해 고객이 Term deposit(정기 예금)에 가입했는지 Classification 한다.




In [1]:
import pandas as pd

col_names = ["age", "job", "marital", "education", "default", "balance", 
             "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome", "y"]

with open('/content/sample_data/bank-full.csv', 'r') as file: 
  bank_data = pd.read_csv(file, sep = ";")

bank_data.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


## 2. Data preparation
> Data를 알고리즘에 따라 학습시키기 이전에, 전처리가 필요하다.

- Categorical data
  - 수치적 계산을 해야하기 때문에, categorical data로는 학습할 수 없다.
  - 따라서 factorize 함수를 통해 명목 데이터를 모두 수치값으로 변경하도록 한다.
- Missing value
  - NaN 값이 dataframe에 존재한다면, 학습이 불가능하다.
  - 따라서 반드시 NaN 값을 확인 해주어야 하며, 처리를 해주어야 한다.
  - 아래와 같은 방법으로 처리가 가능하다.
    - dropna : 해당 행 혹은 열을 삭제한다.
    - fillna : 해당 값을 앞이나 뒤의 값과 동일하게 채운다.
    - interporlate : 보간법을 이용해서 해당 값을 채운다.

In [2]:
## Categorical data를 Numeric data 변환한다.

bank_data['job'], _ = bank_data['job'].factorize()
bank_data['marital'], _ = bank_data['marital'].factorize()
bank_data['education'], _ = bank_data['education'].factorize()
bank_data['default'], _ = bank_data['default'].factorize()
bank_data['housing'], _ = bank_data['housing'].factorize()
bank_data['loan'], _ = bank_data['loan'].factorize()
bank_data['contact'], _ = bank_data['contact'].factorize()
bank_data['month'], _ = bank_data['month'].factorize()
bank_data['poutcome'], _ = bank_data['poutcome'].factorize()
bank_data['y'], _ = bank_data['y'].factorize()

bank_data.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,0,0,0,0,2143,0,0,0,5,0,261,1,-1,0,0,0
1,44,1,1,1,0,29,0,0,0,5,0,151,1,-1,0,0,0
2,33,2,0,1,0,2,0,1,0,5,0,76,1,-1,0,0,0
3,47,3,0,2,0,1506,0,0,0,5,0,92,1,-1,0,0,0
4,33,4,1,2,0,1,1,0,0,5,0,198,1,-1,0,0,0


In [5]:
# 현재 Data에는 nan값이 없음을 알 수 있다.
print(bank_data.isna().sum())

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


## 3. Data classification
이제 실제 Classification을 위한 학습과 예측을 진행한다.  
Classification의 진행 순서는 아래와 같다.
1. 가장 먼저 Train set과 Test set을 분리한다.
  - 이 과정에서 Validation이 진행되면 좋을 것이다.
  - k-fold cross validation을 하듯, 같은 크기로 자르되 우선 first fold로 예측을 해본다.
2. Train set으로 fit() 함수를 통해 학습을 진행한다.
3. 이후 predict() 함수를 통해 실제 Test set을 대상으로 예측을 진행한다.
4. sklear의 metrics 모듈을 통해 모델의 성능을 평가할 수 있다.
  - Accuracy : TP + TN / (TP + TN + FP + FN)
  - Recall : TP  / (TP + FN)
  - Precision : TP / (TP + FP)
  - F1 Score : 2 X (Recall X Precision) / (Recall + Precision)
    - 이는 Recall과 Precision의 조화평균이다.
    - Precision과 Recall이 모두 높아야 높아지는 값이다.
  - Confusion matrix : 실제 TP, TN, FP, FN의 값을 4분면으로 표현한다.

### 3-1. Why use that scoring method?
내가 이번 학습에 사용한 것은 __"Two class classification"__ 모델이다.  
해당 모델은 __두 개의 Class 밖에 없기에__ 결과에 대해 아래와 같은 4가지 평가가 가능하다.
- True Positive
- True Negative
- False Positive
- False Negative

위의 4가지 결과를 가지고 측정할 수 있는 성능의 지표(Metric)는 위에서 언급한대로 아래와 같다.
- Accuracy : TP + TN / (TP + TN + FP + FN)
- Recall : TP  / (TP + FN)
- Precision : TP / (TP + FP)

위의 지표들은 Two class classification 모델 평가에 적합한 성능 지표이기 때문에, 해당 scoring 방식을 사용하게 되었다.

In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import sklearn.metrics as met
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from subprocess import check_call

clf = DecisionTreeClassifier()
train_data = bank_data[col_names[0:16]]
train_label = bank_data['y']

# 예측을 위해 Train set / Test set 분리
# 항상 같은 결과를 유지하기 위해 random_state 값을 부여하도록 한다.
x_train, x_test, y_train, y_test = train_test_split(train_data, train_label, test_size = 0.2, random_state=156)

# Train data로 학습을 시킨 후, Test data로 예측
clf.fit(x_train, y_train)
pred = clf.predict(x_test)

accuracy = met.accuracy_score(y_test, pred)
recall = met.recall_score(y_test, pred)
precision = met.precision_score(y_test, pred)
f1_score = met.f1_score(y_test, pred)
matrix = met.confusion_matrix(y_test, pred)

print('Accuracy: ', format(accuracy,'.2f'),'\n')
print('Recall: ', format(recall,'.2f'),'\n')
print('Precision: ', format(precision,'.2f'),'\n')
print('F1_score: ', format(f1_score,'.2f'),'\n')
print('Confusion Matrix:','\n', matrix)


# Decision Tree의 가시화를 위한 Code
#tree.export_graphviz(clf, 
#               out_file = './tree_model.dot',
#               class_names=['no', 'yes'],
#               feature_names=col_names[0:16],
#               impurity=True, filled=True,
#               rounded=True)

#check_call(['dot', '-Tpng', 'tree_model.dot', '-o', 'DecisionTree.png'])

Accuracy:  0.88 

Recall:  0.48 

Precision:  0.46 

F1_score:  0.47 

Confusion Matrix: 
 [[7421  579]
 [ 547  496]]


## 4. Cross validation
임의로 Test set 20%, Train set 80%로 자른 data set은 위와 같은 학습 결과를 보였다.  
하지만 임의로 자른 Test set이 최적의 set이라는 보장이 없다.  
따라서 우리는 k-fold cross validation을 통해 최적의 Test set을 찾아야 한다.  
k-fold cross validation은 아래와 같이 진행한다.
1. k값을 정한다. (보통은 5 혹은 10을 사용한다)
2. 정확히 k등분한 fold 중 하나를 Test set, 나머지를 Train set으로 잡는다.
3. 학습을 한 후 위와 같이 결과 점수를 매긴다.
  - Accuracy와 F1 Score가 모두 높은 Test set이 최적의 set이라고 할 수 있겠다.

In [36]:
from sklearn.model_selection import KFold
# 5-fold cross validation 진행
clf = DecisionTreeClassifier()
k_fold = KFold(n_splits=5)
iter = 0

feature = bank_data[col_names[0:16]]
label = bank_data['y']

for train_index, test_index in k_fold.split(feature):
    x_train, x_test = feature.iloc[train_index], feature.iloc[test_index]
    y_train, y_test = label[train_index], label[test_index]

    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)
    iter += 1

    accuracy = met.accuracy_score(y_test, pred)
    recall = met.recall_score(y_test, pred)
    precision = met.precision_score(y_test, pred)
    f1_score = met.f1_score(y_test, pred)
    matrix = met.confusion_matrix(y_test, pred)

    train_size = x_train.shape[0]
    test_size = x_test.shape[0]

    print(iter, 'th fold test Accuracy : ', format(accuracy, '.3f'))
    print(iter, 'th fold test Recall : ', format(recall, '.3f'))
    print(iter, 'th fold test Precision : ', format(precision, '.3f'))
    print(iter, 'th fold test F1 Score : ', format(f1_score, '.3f'))
    print(iter, 'th fold Confusion matrix')
    print('\n', matrix, '\n')
    print(iter, 'th Test set index : ', test_index, '\n')
    print('Train set size : ', train_size, ',  Test set size : ', test_size, '\n')

1 th fold test Accuracy :  0.865
1 th fold test Recall :  0.513
1 th fold test Precision :  0.127
1 th fold test F1 Score :  0.204
1 th fold Confusion matrix

 [[7670 1069]
 [ 148  156]] 

1 th Test set index :  [   0    1    2 ... 9040 9041 9042] 

Train set size :  36168 ,  Test set size :  9043 

2 th fold test Accuracy :  0.790
2 th fold test Recall :  0.508
2 th fold test Precision :  0.135
2 th fold test F1 Score :  0.214
2 th fold Confusion matrix

 [[6884 1650]
 [ 250  258]] 

2 th Test set index :  [ 9043  9044  9045 ... 18082 18083 18084] 

Train set size :  36169 ,  Test set size :  9042 

3 th fold test Accuracy :  0.748
3 th fold test Recall :  0.472
3 th fold test Precision :  0.117
3 th fold test F1 Score :  0.187
3 th fold Confusion matrix

 [[6504 1983]
 [ 293  262]] 

3 th Test set index :  [18085 18086 18087 ... 27124 27125 27126] 

Train set size :  36169 ,  Test set size :  9042 

4 th fold test Accuracy :  0.650
4 th fold test Recall :  0.568
4 th fold test Precis

## 5. Optimization
하지만 내가 생각하기엔, 위의 결과가 생각보다 만족스럽지 않다.  
Accuracy가 높으면 F1 score가 낮고, F1 score가 높으면 Accuracy가 높지 않다.  
과연 어떻게 성능을 더 최적화 시킬 수 있을까?  
  
내가 선택한 학습 방법은 Decision Tree를 통한 Classification 이었다.  
생각해보면 Decision Tree의 분기와 깊이에 따라서도 성능이 달라진다.  
- Decision Tree의 분기가 너무 많아지면 Overfitting되어 성능이 저하될 수 있다.
- Decision Tree의 depth가 너무 깊어지면, Overfitting되어 성능이 저하될 수도 있다.  
  
나는 그 중에서 depth를 너무 깊게 내려가지 않도록 max_depth 값을 조절했다.

In [37]:
from sklearn.model_selection import KFold
# 5-fold cross validation 진행을 다시 하는데,
# Max depth를 8로 제한을 두어 너무 깊게 내려가지 않도록 한다.
clf = DecisionTreeClassifier(max_depth = 8)
k_fold = KFold(n_splits=5)
iter = 0

feature = bank_data[col_names[0:16]]
label = bank_data['y']

for train_index, test_index in k_fold.split(feature):
    x_train, x_test = feature.iloc[train_index], feature.iloc[test_index]
    y_train, y_test = label[train_index], label[test_index]

    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)
    iter += 1

    accuracy = met.accuracy_score(y_test, pred)
    recall = met.recall_score(y_test, pred)
    precision = met.precision_score(y_test, pred)
    f1_score = met.f1_score(y_test, pred)
    matrix = met.confusion_matrix(y_test, pred)

    train_size = x_train.shape[0]
    test_size = x_test.shape[0]

    print(iter, 'th fold test Accuracy : ', format(accuracy, '.3f'))
    print(iter, 'th fold test Recall : ', format(recall, '.3f'))
    print(iter, 'th fold test Precision : ', format(precision, '.3f'))
    print(iter, 'th fold test F1 Score : ', format(f1_score, '.3f'))
    print(iter, 'th fold Confusion matrix')
    print('\n', matrix, '\n')
    print(iter, 'th Test set index : ', test_index, '\n')
    print('Train set size : ', train_size, ',  Test set size : ', test_size, '\n')

1 th fold test Accuracy :  0.944
1 th fold test Recall :  0.724
1 th fold test Precision :  0.344
1 th fold test F1 Score :  0.467
1 th fold Confusion matrix

 [[8320  419]
 [  84  220]] 

1 th Test set index :  [   0    1    2 ... 9040 9041 9042] 

Train set size :  36168 ,  Test set size :  9043 

2 th fold test Accuracy :  0.932
2 th fold test Recall :  0.506
2 th fold test Precision :  0.412
2 th fold test F1 Score :  0.454
2 th fold Confusion matrix

 [[8167  367]
 [ 251  257]] 

2 th Test set index :  [ 9043  9044  9045 ... 18082 18083 18084] 

Train set size :  36169 ,  Test set size :  9042 

3 th fold test Accuracy :  0.920
3 th fold test Recall :  0.431
3 th fold test Precision :  0.371
3 th fold test F1 Score :  0.399
3 th fold Confusion matrix

 [[8082  405]
 [ 316  239]] 

3 th Test set index :  [18085 18086 18087 ... 27124 27125 27126] 

Train set size :  36169 ,  Test set size :  9042 

4 th fold test Accuracy :  0.769
4 th fold test Recall :  0.555
4 th fold test Precis

확실히 향상된 성능을 볼 수 있다.  

하지만 depth를 너무 얕게 설정을 하면, 학습 결과에 신뢰도가 떨어지게 될 것이다.  
적절한 depth 또한 여러번의 validation을 통해 찾아야 할 것 같다.  